In [11]:

%load_ext autoreload
%autoreload 2
from rioplatense_hs.openai import get_completion


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
import pandas as pd

pd.options.display.max_colwidth = 1000
#df_train.loc[df_train["LGBTI"] > 0, ["title", "text", "text_label"]].sample(20)

In [13]:
import pandas as pd
from glob import glob

files = glob("../data/test*pred*1shot.csv")

dfs = [pd.read_csv(f) for f in files]


df_test = pd.concat(dfs)

df_test.shape


(5670, 25)

## Error analysis

In [14]:
from rioplatense_hs.preprocessing import text_to_label, labels
from sklearn.metrics import classification_report

pred_labels = [f"PRED_{label}" for label in labels]
gold_labels = df_test[labels]
preds = df_test[pred_labels].astype(int)


print(classification_report(gold_labels, preds, target_names=labels))

              precision    recall  f1-score   support

       WOMEN       0.26      0.31      0.28       131
       LGBTI       0.51      0.45      0.48        88
      RACISM       0.58      0.63      0.60       230
       CLASS       0.25      0.24      0.24        76
    POLITICS       0.14      0.32      0.20       144
    DISABLED       0.33      0.09      0.14        66
  APPEARANCE       0.57      0.37      0.45       189
    CRIMINAL       0.41      0.42      0.42       185

   micro avg       0.37      0.40      0.38      1109
   macro avg       0.38      0.35      0.35      1109
weighted avg       0.41      0.40      0.39      1109
 samples avg       0.07      0.07      0.07      1109



/home/jmperez/projects/rioplatense_hate_speech/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jmperez/projects/rioplatense_hate_speech/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [15]:
pred_labels = [l for l in df_test.columns if l.startswith("PRED")]

label_hate = df_test[labels].sum(axis=1) > 0
pred_hate = df_test[pred_labels].sum(axis=1) > 0

print(classification_report(label_hate, pred_hate))

              precision    recall  f1-score   support

       False       0.91      0.87      0.89      4794
        True       0.43      0.53      0.48       876

    accuracy                           0.82      5670
   macro avg       0.67      0.70      0.68      5670
weighted avg       0.84      0.82      0.83      5670



In [7]:
# Just keep the most important characteristics
core_labels = ["WOMEN", "LGBTI", "RACISM", "CLASS"]
pred_core_labels = [f"PRED_{label}" for label in core_labels]

label_core_hate = df_test[core_labels].sum(axis=1) > 0
pred_core_hate = df_test[pred_core_labels].sum(axis=1) > 0

print(classification_report(label_core_hate, pred_core_hate))


              precision    recall  f1-score   support

       False       0.95      0.94      0.95      3122
        True       0.45      0.50      0.47       280

    accuracy                           0.91      3402
   macro avg       0.70      0.72      0.71      3402
weighted avg       0.91      0.91      0.91      3402



In [10]:
pd.options.display.max_colwidth = 1000

df_test[pred_core_hate != label_core_hate][["context_tweet", "text", "pred_cot", ]].sample(5)

,context_tweet,text,pred_cot
139,El sorteo del Mundial 2023: los Pumas buscan evitar el grupo de la muerte https://t.co/IHkMAikGdG https://t.co/bTQsPk9Cja,"@usuario Seguramente buscarán un grupo donde no estén los ALL BLACKS, detestan a los NEGROS","Desea evitar que el equipo de los All Blacks, que en este contexto se refiere al equipo nacional de rugby de Nueva Zelanda, esté en el mismo grupo que los Pumas. Además, utiliza un término despectivo y discriminatorio (""NEGROS"") para referirse a ellos. La respuesta final es ""racismo""."
727,Coronavirus: Ginés reveló cuál fue la palabra que detonó el acuerdo con Pfizer https://t.co/lIi4Z3eWlE https://t.co/zgd8MCCrui,"@usuario La verdad,ni vale la pena q salgan a explicar nada,si a los mandriles solo se los tranquiliza con bananas🤦🏿‍♂️🤦🏿‍♂️🤦🏿‍♂️","Se utiliza un lenguaje despectivo y racista al referirse a las personas como ""mandriles"" y al hacer referencia a la necesidad de tranquilizarlos con bananas. La respuesta final es ""racismo""."
661,Cristina Kirchner está en los tribunales de Lomas de Zamora para ver las pruebas del espionaje ilegal que sufrió en el gobierno de Macri https://t.co/YR2oW0uNFM,@usuario VIEJA CHOTA PSIQUIÁTRICA ELLA SOLA SE CREE SUS MENTIRAS BASTA DE RELATO!!HASTA CUANDO SEGUIRA ESTA LOCA MANEJANDO EL PAIS Y A LOS JUECES DE LOS HUEVOS SIN DIGNIDAD!JURY O RENUNCIEN SI NO VAN A HACER Q SE CUMPLA LA LEY !\nBASTA DE IMPUNIDAD A LA CHORRA Y SU BANDA K!,"El mensaje contiene expresiones intensas y de rechazo hacia Cristina Kirchner, una persona política, por lo que se puede considerar discurso de odio hacia su ideología política. La respuesta final es ""política"""
568,"Martha Rosenberg: “En situación de pandemia, legalizar el aborto es más urgente que nunca” | Por Mariana Fernández Camacho https://t.co/drwsysry27",@usuario A estos lúmpenes no les importa nada. Son la desgracia de la civilización.,"Desprecio hacia un grupo de personas (lúmpenes), por lo que la respuesta final es ""clase""."
55,"El mensaje de El Dipy que lo convirtió en tendencia en las redes: “Piensan que porque salí de un barrio humilde, soy kirchnerista o soy peronista” https://t.co/CMwm2t7eJA","@usuario Jajaja, no entras en ninguna categoría papá!!! Ahhh y hacete bien esos dientes dracu dracu que tenes! Otario","Insulta y se burla del aspecto físico de la persona mencionada. La respuesta final es ""aspecto""."


In [58]:
pd.options.display.max_colwidth = 1000

df_test[pred_core_hate != label_core_hate][["context_tweet", "text", "pred_cot", ]].sample(200).to_csv("../data/errors.csv")

In [56]:
# LGBTI errors

label = "LGBTI"
df_test.loc[df_test[label] != df_test[f"PRED_{label}"]][["context_tweet", "text", "pred_cot"]].sample(5)

,context_tweet,text,pred_cot
125,😱🔥[¡INESPERADO!]😱🔥 Luis Novaresio recomendó un juguete sexual en las redes &gt;&gt;&gt; El periodista compartió en sus redes sociales una peculiar noticia sobre un objeto para el placer masculino que se volvió viral. ¡Entrá a la nota y mirá! https://t.co/U8NuqEKeQX,"@usuario Alto sida, tiene el viejo esté !!!","El texto utiliza la expresión ""alto sida"" de forma despectiva hacia el periodista. La respuesta final es ""discapacidad""."
532,Pedro Sánchez: “Argentina es hoy una nación más feminista” https://t.co/5s89M2akC5,@usuario Hace mucho luego ya eran Mariconsitos,"El texto utiliza un término ofensivo para referirse a los argentinos y los califica de ""mariconsitos"", lo cual constituye un discurso de odio hacia la comunidad LGBTI. La respuesta final es ""lgbti""."
118,Mara Gómez: la historia de la primera futbolista trans en el torneo argentino https://t.co/rBx3pMPUbB https://t.co/l5I1j0xFku,@usuario Por fin fútbol femenino de calidad,"El comentario alude a la calidad del fútbol femenino, pero no contiene ninguna expresión de odio hacia las características protegidas. La respuesta final es 'nada'."
181,"Contó que era lesbiana, su papá le confesó que era gay y ahora su madre se enamoró de una mujer, lo que inspiró su segundo film | Por Hugo Martin\nhttps://t.co/dt3NtrVJIW","@usuario Los comentarios dios, están todos en el closet odiando a la comunidad","El texto alude a que aquellos que hacen comentarios negativos hacia la comunidad LGBTQ+ están en el closet. La respuesta final es ""lgbti"""
844,"Mariana Genesio Peña pasa su cuarentena total con guantes, barbijo y desnuda: “Mi cuerpo es el planeta Tierra” https://t.co/igQITEEaKl",@usuario Guarda con el paquete...,"El texto hace referencia al cuerpo desnudo de Mariana Genesio Peña, pero no contiene un discurso de odio o comentarios negativos hacia ella. La respuesta final es 'nada'."


In [80]:
df_train[(df_train["POLITICS"] > 0) & (df_train["num_hatred"] > 1)][["context_tweet", "text", "text_label"]].sample(5)

,context_tweet,text,text_label
31940,"Grabois defendió la toma de tierras: ""El que dice que ocupar es delito, miente "" https://t.co/qXZsHYnmZ3 https://t.co/R2BLqqV23a",@usuario Yo le pongo una dinamita a estos vagos,"clase, política"
16559,"Le sacó el arma al ladrón que lo asaltó, lo corrió y lo mató de un disparo: quedó detenido https://t.co/fU6UWLDqQe","@usuario Cagones hoy marchen para que suelten a este hombre, no por el hijo de puta de Macri que nos fundió y ayudó a que vuelvan estos negros hijos de puta kirchnerista","racismo, política, criminal"
7604,Santa Fe: rompen más silobolsas y crece la preocupación en el campo https://t.co/Pb1gaBqyXU https://t.co/cPlGjy3V6T,@usuario Donde quedaron los escopetazos Argentina mia?...,"política, criminal"
34614,"Juan Grabois: ""Hay una oleada de toma de tierras y se va a profundizar "" https://t.co/2B1JL2BkHh https://t.co/jJXbT0X564","@usuario Si es una amenaza!!!!??!!...a Vos hdrmPtas, seras otro de los tantos que vamos a LINCHAR en la Plaza de Mayo!!!!","clase, política"
21056,"“Tenemos explotados los teléfonos con denuncias por gente que no cumple la cuarentena”, aseguró la titular de Migraciones | Por Hugo Martin https://t.co/IJgFwW4Gbk","@usuario Se supone q es gente escolarizada, con educación, después dicen peronchos! Hdp","racismo, política"


In [91]:
sample_df["PRED_HATEFUL"] = pred_hate.astype(int)

In [92]:
sample_df.to_csv("../data/sample_dev_df.csv", index=False)